В этой работе мы реализуем переводчик с русского языка на язык Старшей Речи и наоборот. Тоже самое с английским.
Мы реализуем алгоритм для перевода с русского на Старшую Речь.
Далее используем его для формирования примерно 200 эталонных переводов, на основе которых будем оценивать качество переводов следующих моделей:
1. Llama
2. Deepseek
3. Gemma

Перед началом переводов каждая модель будет настроена промптами.

In [5]:
import pandas as pd

import requests
import re
import string

import pymorphy2
lemmer = pymorphy2.MorphAnalyzer()

import ollama
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [6]:
df = pd.read_csv('../lr3/witcher_words.csv')

Напишем прямой алгоритм перевода с русского на Старшую Речь.

In [24]:
def translate_russian_to_elvish(text: str) -> str:
    """
    Переводит русский текст на язык Старшей Речи.
    Слова, перевод которым подобрать не получилось, латинизируются.
    Присутствует обработка некоторых устойчивых выражений из словаря.
    """

    # Унифицируем е/ё:
    text = text.replace('ё', 'е')

    # Результирующие подстроки для склеивания в конце.
    result_subs = []

    # Отделяем слова от пунктуационных знаков:
    subs = re.split(r'(?<=\.|!|\?|,|;|:|\s|\n|\(|\))|(?=\.|!|\?|,|;|:|\s|\n|\(|\))', text)

    next_i = 0
    for i, sub in enumerate(subs):
        if i < next_i:
            continue

        next_i += 1
        if sub in string.punctuation + ' \n':
            result_subs.append(sub)
            continue

        word = sub
        is_title = word.replace('\'', '').istitle()
        word = word.lower()
        word = _word_to_lemm(word)

        # Текущее слово может быть началом какого-то устойчивого выражения, проверим это.
        it_is_real_phrase = False
        phrases = _find_phrases_that_starts_with_russian_word(word)
        if phrases and i != len(subs) - 1:
            # Будем прибавлять сюда последующие пробелы и слова, пока не встретим что-то лишнее либо не убедимся
            # в том, что это выражение - действительная устойчивая фраза из словаря.
            summ_word = word

            # Количество итераций для пропуска во внешнем цикле (через `next_i`)
            count_to_pass = 0

            for sub in subs[i + 1:]:
                if sub in '.!?':
                    break

                count_to_pass += 1
                if sub not in string.punctuation + ' ':
                    sub = _word_to_lemm(sub.lower())
                elif sub == ',':
                    # Опускаем запятые между словами потенциальной фразы.
                    sub = ''

                summ_word += sub

                need_continue = False
                if summ_word in phrases:
                    it_is_real_phrase = True
                    break

                for phrase in phrases:
                    if summ_word in phrase:
                        need_continue = True
                        break

                if not need_continue:
                    break

            if it_is_real_phrase:
                word = summ_word
                next_i += count_to_pass
                translation = _try_to_find_russian_word_translation(word)

        if not it_is_real_phrase:
            # Стандартная обработка одиночного слова с подбором синонимов через сайт.
            translation = None
            try:
                translation = _try_to_find_russian_word_translation(word)
            except ValueError:
                for synonym in _synonyms_of_word(word):
                    try:
                        translation = _try_to_find_russian_word_translation(synonym)
                    except ValueError:
                        continue

            if translation is None:
                translation = _transliterate_russian_to_latin(word)

        if is_title:
            translation = translation.capitalize()

        result_subs.append(translation)

    return ''.join(result_subs)


def _word_to_lemm(word: str) -> str:
    """Переводит слово/словосочетание в начальную форму."""
    parts = []
    for part in word.split():
        part = lemmer.parse(part)[0].normal_form.replace('ё', 'е')
        parts.append(part)

    return ' '.join(parts)


def _try_to_find_russian_word_translation(word: str) -> str:
    """Ищет перевод русского слова/словосочетания в датасете."""
    result = df.loc[df['translation'] == word, 'text']
    if not result.empty:
        return result.values[0]
    else:
        raise ValueError


def _find_phrases_that_starts_with_russian_word(word: str) -> list[str]:
    return df.loc[df['translation'].str.startswith(word) & df['translation'].str.contains(' '), 'translation'].tolist()


def _synonyms_of_word(word: str) -> str:
    """Выдаёт всевозможные синонимы заданного слова. Использует сетевой запрос."""
    html = requests.get(f'https://text.ru/synonym/{word}').text
    
    match = re.search(r'<meta name=\"description\" content=\"Синонимы к слову [^—]*:([^\"]+)\" />', html)
    if match is None:
        return []

    text = match.group(1)
    synonyms = [synonym.strip() for synonym in text.strip().split('—')]
    synonyms.remove('')

    return synonyms


def _transliterate_russian_to_latin(russian_word: str) -> str:
    """Функция для латинизации русского слова."""
    translit_dict = {
        'а': 'a', 
        'б': 'b', 
        'в': 'v', 
        'г': 'g', 
        'д': 'd', 
        'е': 'e', 
        'ё': 'yo',
        'ж': 'zh', 
        'з': 'z', 
        'и': 'i', 
        'й': 'y', 
        'к': 'k', 
        'л': 'l', 
        'м': 'm',
        'н': 'n', 
        'о': 'o', 
        'п': 'p', 
        'р': 'r', 
        'с': 's', 
        'т': 't', 
        'у': 'u',
        'ф': 'f', 
        'х': 'kh', 
        'ц': 'ts', 
        'ч': 'ch', 
        'ш': 'sh', 
        'щ': 'shch', 
        'ъ': '',
        'ы': 'y', 
        'ь': '', 
        'э': 'e', 
        'ю': 'yu', 
        'я': 'ya',
    }

    latin_word = ''.join(translit_dict.get(symbol, symbol) for symbol in russian_word.lower())
    return latin_word

Посмотрим пример:

In [26]:
print(translate_russian_to_elvish('Здравствуй, Геральт Белый Волк! Твоё присутствие - большая честь для всех нас.'))

Cead, Geralt Gwynbleidd! Do prisutstvie - bolshiy aere а evellienn sinn.


Сформируем списки эталонных переводов.

In [28]:
sequences = []
with open('./sequences.txt', 'r', encoding='utf-8') as f:
    i = 0
    for record in f:
        record = record.strip()
        if not record:
            continue

        if i % 2 == 0:
            # Записываем русское предложение + перевод.
            sequences.append([
                record,
                translate_russian_to_elvish(record),
            ])
        else:
            # Дозаписываем соответствующее английское предложение.
            sequences[-1].append(record)
            print('Одна запись создана.')

        i += 1

sequences


KeyboardInterrupt



Рассмотрим модели LLM для решения задачи перевода. Напишем общий класс для всех наших моделей из Olama.

In [19]:
class Translator:
    """Обёртка над моделью LLM из Olama для реализации перевода."""

    _ONE_REQUEST_RECORD_COUNT = 100

    def __init__(self, name: str) -> None:
        self._name = name
        self._context_messages = []

    def init(self) -> None:
        """Подаёт на вход модели порционно все слова из словаря, после чего можно пользоваться методом `translate`."""

        print('Начало инициализации.')
        self._request('Сейчас я буду отправлять тебе порциями слова на языке Старшей Речи и переводы на русский из словаря. '
                      'Тебе нужно просто запоминать их и отвечать "ok". Потом я отправлю тебе "go" - это означает, что словарь кончился. '
                      'Потом ты сможешь переводить с русского/английского на Старшую Речь и наоборот. '
                      'Если слова нет в словаре, то попробуй подобрать по синониму или самое подходящее по смыслу. '
                      'В самом худшем случае - латинизируй текст.')

        portion = ''
        for record in df.itertuples():
            portion += f'{record[1]} - {record[2]}\n'
            if record[0] % self._ONE_REQUEST_RECORD_COUNT == 0 or record[0] == len(df) - 1:
                print('Новый шаг инициализации...')
                self._request(portion)
                portion = ''

        self._request('go')
        print('Инициализация завершена.')

    def translate_russian_to_elvish(self, text: str) -> str:
        """Переводит русский текст на Старшую Речь."""
        return self._request(f'Переведи русский текст "{text}" на Старшую Речь согласно словарю. Напиши только перевод.')

    def translate_elvish_to_russian(self, text: str) -> str:
        """Переводит текст на языке Старшей Речи на русский."""
        return self._request(f'Переведи текст на языке Старшей Речи "{text}" на русский согласно словарю. Напиши только перевод.')

    def translate_english_to_elvish(self, text: str) -> str:
        """Переводит английский текст на Старшую Речь."""
        text = self._request(f'Переведи английский текст "{text}" на русский. Напиши только перевод.')
        return self.translate_russian_to_elvish(text)

    def translate_elvish_to_english(self, text: str) -> str:
        """Переводит текст на языке Старшей Речи на английский."""
        text = self.translate_elvish_to_russian(text)
        return self._request(f'Переведи русский текст "{text}" на английский. Напиши только перевод.')

    def _request(self, text: str) -> str:
        """Отправляет запрос модели и выдает ответ."""
        self._context_messages.append({
            'role': 'user', 
            'content': text,
        })
        answer = ollama.chat(model=self._name, messages=self._context_messages)['message']['content']
        if '</think>' in answer:
            answer = answer.split('</think>')[1]

        return asnwer.strip()

Начнем тестировать наши модели.

In [27]:
for model_name in ('llama3', 'deepseek-r1', 'gemma3'):
    print(model_name)

    translator = Translator(model_name)
    translator.init()

    bleu_scores_russian_to_elvish = []
    bleu_scores_elvish_to_russian = []

    bleu_scores_english_to_elvish = []
    bleu_scores_elvish_to_english = []

    for record in sequences:
        print('Одна запись обработана.')

        bleu_scores_russian_to_elvish.append(
            sentence_bleu(
                translator.translate_russian_to_elvish(record[0]), 
                record[1],
            )
        )
        bleu_scores_elvish_to_russian.append(
            sentence_bleu(
                translator.translate_elvish_to_russian(record[1]), 
                record[0],
            )
        )

        bleu_scores_english_to_elvish.append(
            sentence_bleu(
                translator.translate_english_to_elvish(record[2]), 
                record[1],
            )
        )
        bleu_scores_elvish_to_english.append(
            sentence_bleu(
                translator.translate_elvish_to_english(record[1]), 
                record[2],
            )
        )

    score = sum(bleu_scores_russian_to_elvish) / len(bleu_scores_russian_to_elvish)
    print('Russian-To-Elvish Score:', score)

    score = sum(bleu_scores_elvish_to_russian) / len(bleu_scores_elvish_to_russian)
    print('Elvish-To-Russian Score:', score)

    score = sum(bleu_scores_english_to_elvish) / len(bleu_scores_english_to_elvish)
    print('English-To-Elvish Score:', score)

    score = sum(bleu_scores_elvish_to_english) / len(bleu_scores_elvish_to_english)
    print('Elvish-To-English Score:', score)

llama3
Начало инициализации.



KeyboardInterrupt



Вывод: Модель <...> показала себя лучше всего с показателем <...> => выбираем её.
В целом, перевод с русского на Старшую Речь можно выполнять напрямую через эталонный алгоритм (в начале файла), а остальные варианты задачи делать уже через выбранную модель.
